In [0]:

import numpy as np
import pandas as pd
from imblearn.pipeline import Pipeline as imPipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.base import TransformerMixin, BaseEstimator
import spacy
from tqdm import tqdm


In [0]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=34c30daa2d32a2c822e7aa82ef9ded328192992d0f4535f784afd3ad5608619f
  Stored in directory: /tmp/pip-ephem-wheel-cache-6j7g65ph/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
from google.colab import drive
drive.mount('/content/drive')
#For Google colab only

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv("/content/drive/My Drive/notebooks/new_clean_2million.csv")
df = df[df['reviewText'].notna()]
#df = df.dropna()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Split data

In [0]:
X = df.reviewText.values
y = df.overall.values

In [0]:
num = len(df)
X, y  = X[:num], y[:num]
le = LabelEncoder()
y = le.fit_transform(y)

In [0]:
# vectoriser = TfidfVectorizer(analyzer='word', max_features=5000)
# X = vectoriser.fit_transform(X)
# rus = RandomUnderSampler()
# X_rus, y_rus = rus.fit_sample(X, y)

In [0]:

def train_predict(pipline):
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    score = classification_report(y_test, y_pred)
    #score = roc_auc_score(y_test, y_pred, average="macro")
    print(score)


Embeddings Converter


In [0]:
#!python -m spacy download en_core_web_lg
disable = ['textcat', 'ner', 'parser']
nlp= spacy.load("/content/drive/My Drive/notebooks/la_vectors_wiki_lg", disable=disable)
#nlp = spacy.load("en_core_web_lg", disable=disable)


In [0]:
disable = ['textcat', 'ner', 'parser']
nlp = spacy.load("en_core_web_lg", disable=disable)

In [0]:
vectors = np.empty(shape=(len(df),300))
missed = 0
for inx,text in enumerate(tqdm(X)):
  try:
    vectors[inx] = nlp(text).vector
  except:
    missed +=1


  4%|▍         | 99577/2461627 [09:23<3:50:47, 170.58it/s]

In [0]:
#X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.33, random_state=42)

In [0]:
def make_balance_pipe(steps):
    
    
    x = list(range(len(steps)))
    pipeline= [(str(x),y) for x,y in zip(x,steps)]

    return imPipeline(pipeline)


Naive Bayes - does not work with vector embeddings


Max Entropy

In [0]:
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [0]:

max_ent = Pipeline([
    ('classifier', LogisticRegression(penalty='l2', C=1.0)),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', max_ent)
])

In [0]:
train_predict(pipeline)

/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.20957725946613334


Logistic Regression

In [0]:
params={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
logistic_regression = Pipeline([
    ('classifier', GridSearchCV(LogisticRegression(), params, cv=10)),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', logistic_regression)
])

In [0]:
train_predict(pipeline)




### Bagging Models

Random Forest

In [0]:
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [0]:
# random_forest = Pipeline([
#     ('classifier', RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid, 
#                                       n_iter = 100, cv = 3, verbose=2, random_state=42, 
#                                       n_jobs = -1)),
# ])

random_forest = Pipeline([
    ('classifier', RandomForestClassifier()),
])

pipeline = Pipeline([
    ('classifier', random_forest)
])

In [0]:
train_predict(pipeline)

### Boosting Models

XGBoost


In [0]:
# !wget https://s3-us-west-2.amazonaws.com/xgboost-wheels/xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl
# !pip uninstall xgboost --yes
# !pip install xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl

# For Google Colab Only

In [0]:
from xgboost import XGBClassifier

In [0]:
xgb_params = {'eta': 0.3,
              'max_depth': 5,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              'objective': 'binary:logistic',
              'eval_metric': 'auc',
              'seed': 23,
               'tree_method':"gpu_hist"
             }
params = {
    'max_depth': 3,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 5,
    'n_gpus': 1,
    'tree_method':"gpu_hist"
}

xg_boost = Pipeline([
    ('classifier', XGBClassifier())
])

pipeline = Pipeline([
    ('classifier', xg_boost)
])


In [0]:
train_predict(pipeline)

              precision    recall  f1-score   support

           0       0.53      0.72      0.61       253
           1       0.43      0.25      0.32       183
           2       0.28      0.15      0.20       123
           3       0.42      0.24      0.31       174
           4       0.69      0.88      0.77       416

    accuracy                           0.57      1149
   macro avg       0.47      0.45      0.44      1149
weighted avg       0.53      0.57      0.53      1149



AdaBoost


In [0]:
from sklearn.ensemble import AdaBoostClassifier


In [0]:

adaBoost = Pipeline([
    ('classifier', AdaBoostClassifier()),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', adaBoost)
])


In [0]:
train_predict(pipeline)


Voting classifier

In [0]:
from sklearn.ensemble import VotingClassifier



In [0]:
models = [
    MultinomialNB(),
    BernoulliNB(),
    LogisticRegression(),
    SGDClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier()
]

m_names = [m.__class__.__name__ for m in models]

models = list(zip(m_names, models))
vc = VotingClassifier(estimators=models)

In [0]:



votingClassifier = Pipeline([
    ('classifier', vc),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', votingClassifier)
])




In [0]:
train_predict(pipeline)

/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
pipeline = make_balance_pipe([TfidfVectorizer(),RandomUnderSampler(),vc])
train_predict(pipeline)

In [0]:
df.reviewText.head()

0    this book had sweet romance  laugh out loud mo...
1    charlie  is having a bad year because she find...
2    this was a great story  the author really deve...
3    tucked away was a sweet story about starting o...
4    what an incredibly sweet contemporary romance ...
Name: reviewText, dtype: object